# POC for NCAABB Elo

In [ ]:
cd ..

In [ ]:
from src.consts import ESPNSportTypes, START_SEASONS, SEASON_GROUPS
from src.utils import find_year_for_season, create_dataframe, put_dataframe, get_dataframe
from src.steps.events_runner import run_events_for_sport
from src.event import ESPNEventsAPI
import numpy as np
import pandas as pd
import os
import datetime

## If its refresh pull whole thing
## If its upsert get the latest seasons in the fs and determine active sports that need pulling

sports = [
    ESPNSportTypes.COLLEGE_BASKETBALL,
    #ESPNSportTypes.COLLEGE_FOOTBALL,
    #ESPNSportTypes.COLLEGE_BASEBALL,
    #ESPNSportTypes.COLLEGE_HOCKEY,
    ESPNSportTypes.COLLEGE_LACROSSE,
    ESPNSportTypes.NFL
    #ESPNSportTypes.NBA

]
import time
status_reports = {}
for sport in sports:
    start = time.time()
    try:
        #run_events_for_sport(root_path='./data/events', sport=sport, espn_events_api=ESPNEventsAPI())
        status_reports[sport] = {
            'status':True,
            'execution_time':round(time.time() - start, 2),
            'end_datetime': datetime.datetime.utcnow()
        }
    except Exception as e:
        print('FAILURE')
        print(e)
        status_reports[sport] = {
            'status':False,
            'execution_time':round(time.time() - start, 2),
            'end_datetime': datetime.datetime.utcnow()
        }
print('')
print('Events Pump Status Report')
print('-'*110)
duration = 0
for key, report in status_reports.items():
    duration = duration + report['execution_time']
    print(f"    {key}: {'PASSED' if report['status'] else 'FAILED'} -- took {report['execution_time']} sec, finished at ({report['end_datetime']}) ")
print('')
print(f'Pump took {duration} sec')
print('-'*110)



In [ ]:
sports = [ESPNSportTypes.COLLEGE_BASKETBALL, ESPNSportTypes.COLLEGE_BASEBALL,ESPNSportTypes.COLLEGE_FOOTBALL,ESPNSportTypes.COLLEGE_HOCKEY]
SCHEMA = {
            'id':np.int64,
            'season':np.int32,
            'is_postseason': np.int8,
            'tournament_id':'Int32',
            'is_finished': np.int8,
            'neutral_site': np.int8,
            'home_team_id':np.int32,
            'home_team_score':'Int32',
            'away_team_id':np.int32,
            'away_team_score':'Int32',
        }
for sport in sports:
    for file in os.listdir(f'./data/events/{sport.value}'):
        fs_df = get_dataframe(f'./data/events/{sport.value}/{file}')
        put_dataframe(fs_df, f'./data/events/{sport.value}/{file}', SCHEMA)




In [ ]:
espn_events_api=ESPNEventsAPI()
team_ids = []
url = 'http://sports.core.api.espn.com/v2/sports/lacrosse/leagues/mens-college-lacrosse/seasons/2008/teams'
res = espn_events_api.api_request(url+'?limit=500')
for item in res['items']:
    team_ids.append(int(item['$ref'].replace(url+'/','').split('?')[0]))
team_ids